In [2]:
import pandas

training_data = pandas.read_csv('data/train.csv')
training_data = training_data.sample(n=training_data.shape[0])

          ID      target  48df886f9  0deb4b6a8  34b15f335  a8cb14b00  \
0  000d6aaf2  38000000.0        0.0          0        0.0          0   

   2f0771a37  30347e683  d08d1fbe3  6ee66e115    ...      3ecc09859  \
0          0          0          0          0    ...            0.0   

   9281abeea  8675bec0b  3a13ed79a  f677d4d13  71b203550  137efaa80  \
0        0.0        0.0          0          0          0          0   

   fb36b89d9  7e293fbaf  9fc776466  
0          0          0          0  

[1 rows x 4993 columns]
             ID      target  48df886f9  0deb4b6a8  34b15f335  a8cb14b00  \
2805  a46af09b5  3142857.14        0.0          0        0.0          0   

      2f0771a37  30347e683  d08d1fbe3  6ee66e115    ...      3ecc09859  \
2805          0          0          0          0    ...            0.0   

      9281abeea  8675bec0b  3a13ed79a  f677d4d13  71b203550  137efaa80  \
2805        0.0        0.0          0          0          0          0   

      fb36b89d9  7e29

In [ ]:
rows, cols = training_data.shape
chunk_size = rows // 10 + 1
for i in range(0, chunk_size, rows):
    validation_subset = training_data.iloc[i:i + chunk_size]
    training_subset = pandas.concat([
        training_data.iloc[0:i],
        training_data.iloc[i + chunk_size:]])
    assert len(training_subset) + len(validation_subset) == rows, (
        'training size {}, validation size {} should add to total size {}'.format(
            len(training_subset), len(validation_subset), len(training_data)))
    break
